# More Naive Bayes

## DSC 40A, Fall 2022

In [1]:
import pandas as pd
import numpy as np

In this notebook we'll look at how to se Naive Bayes to classify emails as spam or ham (not spam).

<br>

<center>
<img src='https://images2.minutemediacdn.com/image/upload/c_crop,h_1576,w_2800,x_0,y_52/v1554931909/shape/mentalfloss/20997-istock-471531747.jpg?itok=3s4MLcXA' width=400></center>

### The data

Let's load in a dataset of real spam and ham emails.

In [2]:
data = pd.read_csv('data/spam_ham_dataset.csv').get(['text', 'label'])

In [3]:
data

text label
0     Subject: enron methanol ; meter # : 988291\r\n...   ham
1     Subject: hpl nom for january 9 , 2001\r\n( see...   ham
2     Subject: neon retreat\r\nho ho ho , we ' re ar...   ham
3     Subject: photoshop , windows , office . cheap ...  spam
4     Subject: re : indian springs\r\nthis deal is t...   ham
...                                                 ...   ...
5166  Subject: put the 10 on the ft\r\nthe transport...   ham
5167  Subject: 3 / 4 / 2000 and following noms\r\nhp...   ham
5168  Subject: calpine daily gas nomination\r\n>\r\n...   ham
5169  Subject: industrial worksheets for august 2000...   ham
5170  Subject: important online banking alert\r\ndea...  spam

[5171 rows x 2 columns]

Here's what an email in our DataFrame looks like:

In [4]:
print(data.get('text').iloc[0])

Subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes .


Let's convert all emails to lower-case.

In [5]:
data['text'] = data['text'].str.lower()

In [6]:
print(data.get('text').iloc[0])

subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes .


### Creating a design matrix

Let's load in a dictionary of words that we can use to differentiate spam and ham emails.

In [7]:
words = ['body', 'click', 'please', 'base64', '2002', 'html', 'subscribed',
         'wrote', 'mortgage', 'align3dcenterfont', 'dear', 'br', 'width10img',
         'divfont', 'im', 'receive', 'list', 'tags', 'web', 'base64', 'click',
         'body', 'please', 'money', 'offer', 'receive', 'contact', 'free',
         'tr', 'removed', 'remove', 'html', 'font', 'form',
         'credit', 'business', 'div']

Before we run Naive Bayes, we need to use the bag-of-words encoding to come up with features.

For a single word, such as `'please'`, we can come up with a column of our design matrix as follows:

In [8]:
data['text'].str.contains('please').astype(int)

0       1
1       0
2       0
3       0
4       0
       ..
5166    0
5167    0
5168    0
5169    0
5170    1
Name: text, Length: 5171, dtype: int64

To do this for every single word, we have:

In [9]:
def featurize(email):
    '''Returns a Series containing the feature vector for a single email.'''
    return pd.Series({word: int(word in email) for word in words})

In [10]:
design_matrix = data['text'].apply(featurize)

`design_matrix` now contains one row per email and one column per word in our dictionary:

In [11]:
design_matrix

body  click  please  base64  2002  html  subscribed  wrote  mortgage  \
0        0      0       1       0     0     0           0      0         0   
1        0      0       0       0     0     0           0      0         0   
2        0      0       0       0     0     0           0      0         0   
3        0      0       0       0     0     0           0      0         0   
4        0      0       0       0     0     0           0      0         0   
...    ...    ...     ...     ...   ...   ...         ...    ...       ...   
5166     0      0       0       0     0     0           0      0         0   
5167     0      0       0       0     0     0           0      0         0   
5168     0      0       0       0     0     0           0      0         0   
5169     0      0       0       0     0     0           0      0         0   
5170     0      1       1       0     0     0           0      0         0   

      align3dcenterfont  ...  contact  free  tr  removed  remove  font  form  \
0                     0  ...        0     0   1        0       0     0     0   
1                     0  ...        0     0   0        0       0     0     0   
2                     0  ...        0     0   1        0       0     0     0   
3                     0  ...        0     0   1        0       0     0     0   
4                     0  ...        0     0   0        0       0     0     0   
...                 ...  ...      ...   ...  ..      ...     ...   ...   ...   
5166                  0  ...        0     0   1        0       0     0     0   
5167                  0  ...        0     0   1        0       0     0     0   
5168                  0  ...        0     0   0        0       0     0     1   
5169                  0  ...        0     0   1        0       0     0     0   
5170                  0  ...        0     0   0        0       0     0     1   

      credit  business  div  
0          0         0    0  
1          0         0    0  
2          0         0    0  
3          0         0    0  
4          0         0    0  
...      ...       ...  ...  
5166       0         0    0  
5167       0         0    0  
5168       0         0    0  
5169       0         0    0  
5170       0         0    0  

[5171 rows x 31 columns]

### Setting up the model

We're now ready to run the Naive Bayes algorithm. We could do this ourselves by hand, but we'll instead use the `CategoricalNB` object from `sklearn` that you'll also use in Homework 8.

In [12]:
from sklearn.naive_bayes import CategoricalNB

Let's create a `CategoricalNB` object. `alpha=1` enables smoothing as we've defined it in class.

In [13]:
model = CategoricalNB(alpha=1)

We now need to "fit" the model to the data.

In [14]:
model.fit(X=design_matrix, y=data['label'])

CategoricalNB(alpha=1)

### Making predictions

Now that we've "fit" the model, we can use it to make predictions.

In [15]:
def get_prediction(email, prob=False):
    '''Calls model.predict to determine the predicted class (spam or ham) for a single email.
       If the optional argument prob=True is used, the probability of the prediction for the more likely class is printed.
    '''
    if prob:
        probs = model.predict_proba(featurize(email).values.reshape(1, -1))
        print(f'Probability: {np.round(max(probs[0]), 4) * 100}%')
    return model.predict(featurize(email).values.reshape(1, -1))[0]

In [16]:
get_prediction('''
my name is king triton
please click on this email to receive free credit cards for your new business
''')

/home/hytruongson/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


'spam'

In [17]:
get_prediction('''
hey! i had a question on homework 8, part 1d in dsc 40a.
''', prob=True)

Probability: 92.96%


/home/hytruongson/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
/home/hytruongson/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


'ham'

### Accuracy

A metric that we often use when classifying is **accuracy**, which is defined as the fraction of data points that were classified correctly.

As it turns out, `sklearn` has a built-in method that calculates accuracy – `.score`.

In [18]:
model.score(X=design_matrix, y=data['label'])

0.7834074647070199

This is telling us that Naive Bayes has an accuracy of 78.34% on the data in `design_matrix`. In practice, however, we'll often care more about the accuracy of a classifer on what is called **test data**, which is data that we didn't use to train the model. (Recall from earlier in the quarter, the purpose of a prediction rule is to make predictions about data for which we don't already know the "right answer".)

### More models

As you know, DSC 40A is called **Theoretical Foundations of Data Science (Part 1)**. That's why we spent most of our time working on paper and talking about the math behind various techniques.

To wrap up the quarter, we'll show you how we can classify emails as spam or ham using a variety of other models in `sklearn`, without walking through the math. You'll see the math in future courses.

**Logistic Regression**

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
lr_model = LogisticRegression()
lr_model.fit(X=design_matrix, y=data['label'])

LogisticRegression()

In [21]:
lr_model.score(X=design_matrix, y=data['label'])

0.8013923805840263

Let's store our accuracies in a dictionary that we can refer back to later.

In [22]:
model_accs = {}
model_accs['naive bayes'] = model.score(X=design_matrix, y=data['label'])
model_accs['logistic regression'] = lr_model.score(X=design_matrix, y=data['label'])

**Decision Trees**

In [23]:
from sklearn.tree import DecisionTreeClassifier

In [24]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X=design_matrix, y=data['label'])

DecisionTreeClassifier()

In [25]:
model_accs['decision tree'] = dt_model.score(X=design_matrix, y=data['label'])
model_accs['decision tree']

0.8632759620963063

**Support Vector Machines**

In [26]:
from sklearn.svm import SVC

In [27]:
svm_model = SVC()
svm_model.fit(X=design_matrix, y=data['label'])

SVC()

In [28]:
model_accs['support vector machine'] = svm_model.score(X=design_matrix, y=data['label'])
model_accs['support vector machine']

0.8458712047959775

**Overall**

In [29]:
model_accs

{'naive bayes': 0.7834074647070199,
 'logistic regression': 0.8013923805840263,
 'decision tree': 0.8632759620963063,
 'support vector machine': 0.8458712047959775}

Note that the code for all three of these new models (Logistic Regression, Decision Trees, and Support Vector Machines) is almost identical. Under the hood, though, there's a lot of math going on. You now have a taste of the math that's involved in making predictions! 😇